# Nose関係メモ

## 必要な知識
### ベルヌーイの定理
$$p_1+\frac{1}{2} \rho v_1^2=p_2+\frac{1}{2}\rho v_2^2$$

つまり
$$v=\sqrt{\frac{2|p_1-p_2|}{\rho}}$$

https://www.cradle.co.jp/glossary/ja_H/detail0038.html

ここで空気密度$\rho$について考える．大気の標準組成はO2:20.99%, N2:78.04%,CO2:0.03%, Ar:0.94%であり，そこから求まる大気の分子量は
$$M=28.966[\mathrm{g/mol}]$$
となる．よって機体の質量密度は，
$$\rho=M\frac{n}{V}=\frac{MP}{RT}$$
ここで，$R=8.314[\mathrm{Nmmol^{-1}K^{-1}}]$，とすると，
$$\rho=\frac{28.966\times P}{8.314\times T}$$
とわかる．


http://sasaki.g1.xrea.com/powerpoint/vaporization-heat/03-Air-density.pdf

In [52]:
#差圧センサ選定用プログラム
import math
M=28.966
R=8.314
P=101325
celsius=20
T=celsius+273.15
#tagert airspeed
v=340*0.5

ro=M*P/(R*T)/1000
# print("ro",ro,"kg/m^3")
P_diff_Pa=(ro*v*v)/2
P_diff_kPa=P_diff_Pa/1000

print("pdiff",P_diff_kPa,"kPa")


pdiff 17.400954329001497 kPa


これを測定できるいい感じの差圧センサはMPXV5050DPだと思う

https://www.digikey.jp/ja/products/detail/nxp-usa-inc/MPXV5050DP/1168433?s=N4IgTCBcDaILIAUAaA1ArABkwEQSAugL5A　

このセンサが測定できる最大大気速度は

In [57]:
#差動圧力
import math
M=28.966
R=8.314
P=101325
celsius=20
T=celsius+273.15
###############
P_diff_kPa=50
###############
P_diff=P_diff_kPa*1000

ro=M*P/(R*T)/1000
v=math.sqrt(2*P_diff/ro)

print("v=",v)


v= 288.16908661510917


よって今回のロケットマッハ0.33を超えていて大丈夫そう．

## ADCの選定

今回用いる5050DPはアナログ値を返すので，それなりのADCが欲しい．でこれ買った

https://akizukidenshi.com/catalog/g/gI-16585/


- 電源電圧：2.5～5.5V
- 消費電流：155μA
- スタンバイ時電流：0.1μA
- 分解能(最低)：12bit
- 分解能(最高)：18bit
- データレート(最低)：3.75sps
- データレート(最高)：240sps
- INL：10ppm
- 内部基準電圧：2.048V
- インターフェース：I2C
- 変換方式：デルタシグマ
- 入力タイプ：差動入力
- 動作温度範囲：-40～125℃
- パッケージ：SOT-23-6

差圧センサは0~50kPaで0~5Vまで出力する．
MAXの5Vのとき，ADCにかかる電圧が2.000V位であってほしい

$$V_{adc}=\frac{R_1}{R_1+R_2}V_{out}$$

のように分圧すればよく，
$$R_1:R_2=V_{adc}:(V_{out}-V_{adc})$$
であることがわかる．

In [58]:
Vadc=2.048
Vout=5

print("R1:R2=1:",Vadc,":",Vout-Vadc)

R1:R2=1: 2.048 : 2.952


とにかく高精度な抵抗を使って2:3を作り出したい．


In [68]:
R1=100
R2=49.9
R=R1+R2
Vout=5
print(R2/R*Vout)

1.6644429619746495


よってGNDよりが $49.9\mathrm{k \Omega}$，差圧センサのVout側が $100\mathrm{k \Omega}$

https://akizukidenshi.com/catalog/g/gR-11804/

https://akizukidenshi.com/catalog/g/gR-11792/